In [1]:
import sqlite3

# Introduzione

In [2]:
# crea un nuovo database su disco
conn = sqlite3.connect('example.db')

# crea un nuovo database in memoria
# conn = sqlite3.connect('example.db')

In [5]:
# chiedi quali sono le tabelle presenti nel database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master")

In [7]:
# il database è vuoto, quindi non ci sono tabelle
cursor.fetchall()

[]

In [8]:
# crea una tabella anagrafica
cursor.execute("CREATE TABLE anagrafica (nome TEXT, cognome TEXT, eta INT)")

In [10]:
cursor.execute("SELECT name FROM sqlite_master")

In [11]:
cursor.fetchall()

[('anagrafica',)]

In [14]:
cursor.execute(
    """
    INSERT INTO anagrafica (nome, cognome, eta)
    VALUES 
        ('Andrea', 'Mercuri', 49),
        ('Alessandra', 'Rossi', 40),
        ('Monica', 'Bianchi', 30)
    """
)

In [15]:
# commit delle modifiche
conn.commit()

In [16]:
# Seleziona tutte le righe della tabella anagrafica
cursor.execute("SELECT * FROM anagrafica")

In [17]:
cursor.fetchall()

[('Andrea', 'Mercuri', 49),
 ('Alessandra', 'Rossi', 40),
 ('Monica', 'Bianchi', 30)]

In [19]:
# Seleziona tutte le righe della tabella anagrafica
cursor.execute("SELECT * FROM anagrafica")

In [20]:
cursor.fetchone(), cursor.fetchone(), cursor.fetchone()

(('Andrea', 'Mercuri', 49),
 ('Alessandra', 'Rossi', 40),
 ('Monica', 'Bianchi', 30))

In [22]:
# Seleziona tutte le righe della tabella anagrafica
cursor.execute("SELECT * FROM anagrafica")

In [23]:
cursor.fetchmany(2), cursor.fetchmany(2)

([('Andrea', 'Mercuri', 49), ('Alessandra', 'Rossi', 40)],
 [('Monica', 'Bianchi', 30)])

In [28]:
# Chiude la connessione al database
conn.close()

In [29]:
# riapre la connessione al database
conn = sqlite3.connect('example.db')

In [30]:
cursor = conn.cursor()

In [32]:
# Seleziona tutte le righe della tabella anagrafica
cursor.execute("SELECT * FROM anagrafica")

In [33]:
# È possibile iterare sul cursore come alternativa all'invocazione delle fetch
for row in cursor:
    print(row)

('Andrea', 'Mercuri', 49)
('Alessandra', 'Rossi', 40)
('Monica', 'Bianchi', 30)


# I placeholders

Il vantaggio di utilizzare i placeholder piuttosto che le f-string o altri metodi di composizione delle stringhe in Python è che i plaveholder proteggono il codice da attacchi di tipo [SQL injection](https://en.wikipedia.org/wiki/SQL_injection).

In [34]:
# Inserisce una nuova riga nella tabella anagrafica
cursor.execute("INSERT INTO anagrafica (nome, cognome, eta) VALUES ('Mario', 'Neri', 25)")

In [35]:
# commit delle modifiche
conn.commit()

In [36]:
cursor.execute("INSERT INTO anagrafica (nome, cognome, eta) VALUES (?, ?, ?)", ('Luca', 'Verdi', 35))

In [37]:
conn.commit()

In [38]:
cursor.execute("INSERT INTO anagrafica (nome, cognome, eta) VALUES (:nome, :cognome, :eta)", {'nome': 'Paolo', 'cognome': 'Gialli', 'eta': 45})

In [39]:
# commit delle modifiche
conn.commit()

In [44]:
# esempio di attacco SQL injection. Notare che -- è un commento in SQL
cursor.execute("SELECT * FROM anagrafica WHERE nome = '{}'".format("Mario' OR 1=1 --"))

In [45]:
cursor.fetchall()

[('Andrea', 'Mercuri', 49),
 ('Alessandra', 'Rossi', 40),
 ('Monica', 'Bianchi', 30),
 ('Mario', 'Neri', 25),
 ('Luca', 'Verdi', 35),
 ('Paolo', 'Gialli', 45)]

In [47]:
cursor.execute("SELECT * FROM anagrafica WHERE nome = ?", ("Mario' OR 1=1 --",))

In [48]:
cursor.fetchall()

[]

# Connection come context manager

In [50]:
# l'oggetto connessione è un context manager, quindi è possibile utilizzarlo in un blocco with.
# In questo caso se si esce dal blocco with regolarmente (senza eccezioni) il commit viene eseguito automaticamente,
# altrimenti viene eseguito il rollback.
# Attenzione: quando il blocco with termina, la connessione non viene chiusa automaticamente.

with conn:
    conn.execute("INSERT INTO anagrafica (nome, cognome, eta) VALUES ('Giovanni', 'Rosa', 55)")
    conn.execute("INSERT INTO anagrafica (nome, cognome, eta) VALUES ('Giuseppe', 'Viola', 65)")
    conn.execute("INSERT INTO anagrafica (nome, cognome, eta) VALUES ('Giorgio', 'Azzurri', 75)")
    # genera un'eccezione, quindi il blocco with eseguirà il rollback
    1 / 0

ZeroDivisionError: division by zero

In [51]:
# select di tutte le righe della tabella anagrafica
cursor.execute("SELECT * FROM anagrafica")
cursor.fetchall()

[('Andrea', 'Mercuri', 49),
 ('Alessandra', 'Rossi', 40),
 ('Monica', 'Bianchi', 30),
 ('Mario', 'Neri', 25),
 ('Luca', 'Verdi', 35),
 ('Paolo', 'Gialli', 45)]

In [52]:
with conn:
    conn.execute("INSERT INTO anagrafica (nome, cognome, eta) VALUES ('Giovanni', 'Rosa', 55)")
    conn.execute("INSERT INTO anagrafica (nome, cognome, eta) VALUES ('Giuseppe', 'Viola', 65)")
    conn.execute("INSERT INTO anagrafica (nome, cognome, eta) VALUES ('Giorgio', 'Azzurri', 75)")
    # nessuna eccezione, quindi il blocco with eseguirà il commit

In [53]:
# select di tutte le righe della tabella anagrafica
cursor.execute("SELECT * FROM anagrafica")
cursor.fetchall()

[('Andrea', 'Mercuri', 49),
 ('Alessandra', 'Rossi', 40),
 ('Monica', 'Bianchi', 30),
 ('Mario', 'Neri', 25),
 ('Luca', 'Verdi', 35),
 ('Paolo', 'Gialli', 45),
 ('Giovanni', 'Rosa', 55),
 ('Giuseppe', 'Viola', 65),
 ('Giorgio', 'Azzurri', 75)]

In [54]:
conn.execute("INSERT INTO anagrafica (nome, cognome, eta) VALUES ('Giacomino', 'Rosa', 55)")
conn.execute("INSERT INTO anagrafica (nome, cognome, eta) VALUES ('Pierino', 'Viola', 65)")

# rollback delle modifiche
conn.rollback()

In [55]:
cursor.execute("SELECT * FROM anagrafica")
cursor.fetchall()

[('Andrea', 'Mercuri', 49),
 ('Alessandra', 'Rossi', 40),
 ('Monica', 'Bianchi', 30),
 ('Mario', 'Neri', 25),
 ('Luca', 'Verdi', 35),
 ('Paolo', 'Gialli', 45),
 ('Giovanni', 'Rosa', 55),
 ('Giuseppe', 'Viola', 65),
 ('Giorgio', 'Azzurri', 75)]